# **Machine Translation Model**

In [ ]:
#@title Load Imports

import tensorflow as tf
import keras
import subprocess
import os
import pandas as pd
import random
import shutil
import numpy as np
import json
import re
import pathlib

from google.colab import drive, files #if use colab
from tensorflow.nn import relu, tanh, softmax
from tensorflow.lite.python import interpreter
from keras import layers
from keras.models import Model
from keras.preprocessing.text import tokenizer_from_json
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [ ]:
#@title Connect with Google Drive
drive.mount('/content/drive')

In [ ]:
#@title Git Clone
git_dir = '/content/IOH-Chat-App'
git_url = 'https://github.com/bangkit-team/IOH-chat-app.git'

if not os.path.exists(git_dir):
  subprocess.call(['git', 'clone', git_url])

In [ ]:
filedir1 = '/content/IOH-chat-app/MachineLearning/datasets/translation/result/eng-ind.csv'
filedir2 = '/content/IOH-chat-app/MachineLearning/datasets/spam/emails.csv'

In [ ]:
df1 = pd.read_csv(filedir1)
df1

In [ ]:
df2 = pd.read_csv(filedir2)
df2 = df2.drop(columns='spam')
df2 = df2.rename(columns={'text': 'English', 'teks': 'Indonesia'})
df2

In [ ]:
df = pd.concat([df1, df2])
df

In [ ]:
start_mark = '<start>'
end_mark = '<end>'

In [ ]:
class TranslatorDataset():
  def __init__(self, dataframe):
    self.dataframe = dataframe
    self.input_tokenizer = None
    self.target_tokenizer = None
    self._load_data_from_file()

  def _load_data_from_file(self):
    df = self.dataframe

    input_lang = df.English.values
    target_lang = df.Indonesia.values

    return input_lang, target_lang

  def _normalize_and_preprocess(self, text, use_mark=False):
    if use_mark:
      text = text.strip()
      text = ' '.join([start_mark, text, end_mark])
    else:
      text = text.strip()

    return text

  def _tokenize(self, sentences, num_words, maxlen):
    punctuation = '!"#$%&()*+,-./:;=?@[\\]^_`{|}~\t\n'

    tokenizer = Tokenizer(num_words=num_words, filters=punctuation)
    tokenizer.fit_on_texts(sentences)

    sequences = tokenizer.texts_to_sequences(sentences)
    sequences = pad_sequences(
        sequences, maxlen=self.maxlen, padding='post', truncating='post')

    return sequences, tokenizer

  def _create_dataset(self):
    input_lang, target_lang = self._load_data_from_file()

    input_sentence = np.array(
        list(map(lambda x: self._normalize_and_preprocess(x, False), input_lang)))
    
    target_sentence = np.array(
        list(map(lambda y: self._normalize_and_preprocess(y, True), target_lang)))
    
    return input_sentence, target_sentence

  def _load_dataset(self, num_words):
    input_lang, target_lang = self._create_dataset()

    self.maxlen = 20
    self.buffer_size = len(input_lang)

    input_sequences, input_tokenizer = self._tokenize(
        input_lang, num_words, self.maxlen)
    
    target_sequences, target_tokenizer = self._tokenize(
        target_lang, num_words, self.maxlen,)

    return (input_sequences, input_tokenizer), (target_sequences, target_tokenizer)
  
  def get(self, num_words, batch_size):
    input, target = self._load_dataset(num_words)

    input_sequences, self.input_tokenizer = input
    target_sequences, self.target_tokenizer = target

    dataset = tf.data.Dataset.from_tensor_slices((input_sequences, target_sequences))
    dataset = dataset.shuffle(self.buffer_size).batch(batch_size, drop_remainder=True)
    dataset = dataset.cache().prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

    return self.input_tokenizer, self.target_tokenizer, dataset

In [ ]:
num_words = 50000
batch_size = 64

In [ ]:
translator_dataset = TranslatorDataset(df)

input_tokenizer, target_tokenizer, dataset = translator_dataset.get(num_words, batch_size)

In [ ]:
input_batch, target_batch = next(iter(dataset))

In [ ]:
input_batch.shape, target_batch.shape

In [ ]:
input_vocab_size = len(input_tokenizer.index_word) + 1
target_vocab_size = len(target_tokenizer.index_word) + 1
input_maxlen = input_batch.shape[1]
target_maxlen = target_batch.shape[1]

input_maxlen, target_maxlen, input_vocab_size, target_vocab_size

In [ ]:
input_example = input_batch[-1]
input_example

In [ ]:
target_example = target_batch[-1]
target_example

In [ ]:
input_sentence = input_tokenizer.sequences_to_texts([input_example.numpy()])[0]
input_sentence

In [ ]:
target_sentence = target_tokenizer.sequences_to_texts([target_example.numpy()])[0]
target_sentence

In [ ]:
embed_dims = 1000
units = 1024

In [ ]:
class Encoder():
  def __init__(self, input_vocab_size, embedding_dims, units):
    self.units = units
    self.batch_size = batch_size
    self.input_vocab_size = input_vocab_size
    self.embedding_dims = embedding_dims

    self.embedding = layers.Embedding(self.input_vocab_size, self.embedding_dims)
    self.lstm_layer = layers.LSTM(self.units,
                                 return_sequences=True,
                                 return_state=True,
                                 recurrent_initializer='glorot_uniform')

  def call(self, inputs):
    embedding = self.embedding(inputs)
    encoder = self.lstm_layer(embedding)

    return encoder

In [ ]:
class BahdanauAttention(layers.Layer):
  def __init__(self, units):
    super(BahdanauAttention, self).__init__()
    self.w1 = layers.Dense(units, use_bias=True) 
    self.w2 = layers.Dense(units, use_bias=True) 
    self.fd = layers.Dense(1)

  def call(self, query, values):
    query_with_time_axis = tf.expand_dims(query, 1)
    
    score = self.fd(tanh(
        self.w1(query_with_time_axis) + self.w2(values)))

    attention_weights = softmax(score, axis=1)

    context_vector = attention_weights * values
    context_vector = tf.reduce_sum(context_vector, axis=1)

    return context_vector, attention_weights

In [ ]:
class Decoder():
  def __init__(self, output_vocab_size, embedding_dims, units):
    self.units = units
    self.output_vocab_size = output_vocab_size
    self.embedding_dims = embedding_dims

    self.embedding = layers.Embedding(self.output_vocab_size, self.embedding_dims)
    self.lstm_layer = layers.LSTM(self.units,
                                  return_sequences=True,
                                  return_state=True,
                                  recurrent_initializer='glorot_uniform')
    self.attention = BahdanauAttention(self.units)
    self.concat = layers.Concatenate()
    self.dense1 = layers.Dense(self.units, activation=tanh, use_bias=False)
    self.dropout = layers.Dropout(.5)
    self.dense2 = layers.TimeDistributed(layers.Dense(self.output_vocab_size))

  def call(self, inputs, en_outputs, state):
    embedding = self.embedding(inputs)
    dec_outputs, dec_h_state, dec_c_state = self.lstm_layer(
        embedding, initial_state=state)
    
    context_vector, attention_weights = self.attention(
        query=dec_outputs, values=en_outputs)
    
    context_and_rnn_output = self.concat(
        [context_vector, dec_outputs])

    attention_vector = self.dense1(context_and_rnn_output)
    outputs = self.dropout(attention_vector)
    outputs = self.dense2(outputs)

    return outputs

In [ ]:
encoder = Encoder(input_vocab_size, embed_dims, units)
en_outputs, en_h_state, en_c_state = encoder.call(input_batch)

en_outputs.shape, en_h_state.shape, en_c_state.shape

In [ ]:
decoder = Decoder(target_vocab_size, embed_dims, units)
dec_outputs = decoder.call(target_batch, en_outputs, [en_h_state, en_c_state])

dec_outputs.shape

In [ ]:
lr = 0.001
epochs = 30

optimizer = tf.keras.optimizers.Adam(
    learning_rate=lr, beta_1=0.9, beta_2=0.98, epsilon=1e-9)
loss = tf.keras.losses.SparseCategoricalCrossentropy(
       from_logits=True, reduction='none')

In [ ]:
class TranslatorModel():
  def __init__(self, input_vocab_size, 
               target_vocab_size, 
               embed_dims, 
               units, 
               maxlen):
    self.input_vocab_size = input_vocab_size
    self.target_vocab_size = target_vocab_size
    self.embed_dims = embed_dims
    self.units = units
    self.maxlen = maxlen

    self.encoder = Encoder(
        self.input_vocab_size, self.embed_dims, self.units)
    
    self.decoder = Decoder(
        self.target_vocab_size, self.embed_dims, self.units)
  
  def build_model(self):
    en_inputs = layers.Input(shape=(self.maxlen,))

    en_output, en_h_state, en_c_state = self.encoder.call(en_inputs)

    dec_outputs = self.decoder.call(
        en_inputs, en_output, [en_h_state, en_c_state])

    model = Model(inputs=[en_inputs], 
                  outputs=[dec_outputs])
    return model

In [ ]:
translator_model = TranslatorModel(
    input_vocab_size,
    target_vocab_size,
    embed_dims,
    units,
    input_maxlen
)
model = translator_model.build_model()

model.compile(
    optimizer=optimizer,
    loss=loss,
    metrics=['accuracy']
)

In [ ]:
checkpoint_path = 'checkpoint/cp.ckpt'
checkpoint_dir = os.path.dirname(checkpoint_path)

callback_early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='loss',
    patience=3, 
    verbose=1)

callback_checkpoint = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, 
    monitor='loss', 
    verbose=1, 
    save_weights_only=True, 
    save_best_only=True)

callbacks = [callback_early_stopping,
             callback_checkpoint]

model.summary()

In [ ]:
model.fit(dataset,
          epochs=epochs,
          callbacks=callbacks,
          verbose=1)

In [ ]:
saved_model_path  = '/content/drive/MyDrive/Company Case Bangkit/TranslationModel/saved_model'
saved_model_dir = os.path.dirname(saved_model_path)

if not os.path.exists(saved_model_dir):
  shutil.rmtree(saved_model_dir)
  
model.save(saved_model_path)

In [ ]:
input_tokenizer_dir = '/content/drive/MyDrive/Company Case Bangkit/TranslationModel/input_tokenizer.json'

input_tokenizer_json = input_tokenizer.to_json()
with open(input_tokenizer_dir, 'w', encoding='utf-8') as f:
    json.dump(input_tokenizer_json, f, ensure_ascii=False)

files.download(input_tokenizer_dir)

In [ ]:
target_tokenizer_dir = '/content/drive/MyDrive/Company Case Bangkit/TranslationModel/target_tokenizer.json'

target_tokenizer_json = target_tokenizer.to_json()
with open(target_tokenizer_dir, 'w', encoding='utf-8') as f:
    json.dump(target_tokenizer_json, f, ensure_ascii=False)

files.download(target_tokenizer_dir)

In [ ]:
class Translator():
  def __init__(self, model_path, input_tokenizer_json, target_tokenizer_json, maxlen):
    self.model_path = model_path
    self.input_tokenizer_json = input_tokenizer_json
    self.target_tokenizer_json = target_tokenizer_json
    self.maxlen = maxlen

    self._load_model()
    self._load_tokenizer()

  def _load_model(self):
    self.model = tf.keras.models.load_model(self.model_path, compile=True)
  
  def _load_tokenizer(self):
    with open(self.input_tokenizer_json) as f:
      input_json = json.load(f)
      self.input_tokenizer = tokenizer_from_json(input_json)

    with open(self.target_tokenizer_json) as f:
      target_json = json.load(f)
      self.target_tokenizer = tokenizer_from_json(target_json)

  def _normalize_and_preprocess(self, text):
    punctuation = '!"#$%&()*+,-./:;=?@[\\]^_`{|}~\t\n'
    
    text = text.lower().strip()
    text = ''.join((filter(lambda x: x not in punctuation, text)))

    return text

  def __call__(self, sentence):
    index_prediction = list()

    normalize_sentence = self._normalize_and_preprocess(sentence)
    sequences = self.input_tokenizer.texts_to_sequences([normalize_sentence])
    sequences = pad_sequences(
        sequences, maxlen=self.maxlen, padding="post", truncating="post")

    predictions = self.model(sequences)

    for i in predictions[0]:
        index_prediction.append(np.argmax(i))

    marks = [start_mark, end_mark]
    result = self.target_tokenizer.sequences_to_texts([index_prediction])[0]

    result = ' '.join([word for word in result.split(' ') if word not in marks])

    return result

In [ ]:
saved_model_path = '/content/drive/MyDrive/Company Case Bangkit/TranslationModel/saved_model'

translator = Translator(
    saved_model_path,
    input_tokenizer_dir,
    target_tokenizer_dir,
    input_maxlen)

In [ ]:
text_input = 'i like apple'

translate = translator(text_input)
print(translate)

In [ ]:
# converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_path)
# converter.optimizations = [tf.lite.Optimize.DEFAULT]
# converter.target_spec.supported_ops = [
#   tf.lite.OpsSet.TFLITE_BUILTINS,
#   tf.lite.OpsSet.SELECT_TF_OPS
# ]
# converter.experimental_lower_tensor_list_ops = False

# tflite_model = converter.convert()

# tflite_model_file = pathlib.Path('translation.tflite')
# tflite_model_file.write_bytes(tflite_model)

# # files.download('translation.tflite')

In [ ]:
# interpreter = tf.lite.Interpreter('translation.tflite')
# interpreter.allocate_tensors()

# input_details = interpreter.get_input_details()[0]
# output_details = interpreter.get_output_details()[0]

# input_details, output_details

In [ ]:
# sequence = input_tokenizer.texts_to_sequences([text_input])
# pad_seqs = pad_sequences(
#     sequence, maxlen=input_maxlen, padding='post', truncating='post')

# input_data = pad_seqs.astype(np.float32)

# interpreter.set_tensor(input_details['index'], input_data)
# interpreter.invoke()

# predictions = interpreter.get_tensor(output_details['index'])
